In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from tqdm import tqdm
from sklearn import preprocessing

In [10]:
import matplotlib.pyplot as plt
import catboost
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn.model_selection import train_test_split
from nltk.tokenize import TweetTokenizer
import nltk
from nltk.stem.snowball import SnowballStemmer
import string
from string import punctuation
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.preprocessing import MaxAbsScaler
from catboost import CatBoostClassifier
from sklearn.model_selection import GridSearchCV

In [11]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [13]:
data = pd.read_csv(r"C:\Users\User\Desktop\HSE Data Science Hack\train.csv")

In [14]:
#data_2 = pd.read_csv('/kaggle/input/bank-sentiments/bank-sentiment.csv')
#data_bert = pd.read_csv('/kaggle/input/sentiment4/sentiment.csv')

In [64]:
#bank_data = pd.read_csv('/kaggle/input/hataton/HSE Data Science Hack /banki.csv')

In [15]:
data = data.fillna('nans')

In [16]:
data.drop('Unnamed: 0', axis=1, inplace=True)

In [17]:
data['1category'].unique()

array(['Communication', '?', 'Quality', 'Price', 'Safety'], dtype=object)

In [18]:
for i in range(19361):
    if data['2category'][i] == 'nans':
        data['2category'][i] = data['1category'][i]

In [19]:
data

,sentence,1category,2category,sentiment
0,При этом всегда получал качественные услуги.,Communication,Communication,+
1,"Не вижу, за что хотя бы 2 поставить, сервис на 1!",?,?,−
2,"Вот так ""Мой любимый"" банк МКБ меня обманул.",?,?,−
3,Отвратительное отношение к клиентам.,Communication,Communication,−
4,"Всегда в любое время дня и ночи помогут, ответ...",Communication,Communication,+
...,...,...,...,...
19356,Никогда и ни в коем случае не открывайте счет ...,Communication,Communication,−
19357,ТИ откровенно забили на качество и развивают с...,Quality,Quality,−
19358,"Я считаю, это прорыв и лидерство финансовых ус...",?,?,+
19359,"Писал мужчина очень доходчиво, не финансовым я...",Communication,Communication,+


In [20]:
for i in range(19361):
    if data['2category'][i] == '?':
        data['2category'][i] = np.nan
        data['1category'][i] = np.nan

In [21]:
data

,sentence,1category,2category,sentiment
0,При этом всегда получал качественные услуги.,Communication,Communication,+
1,"Не вижу, за что хотя бы 2 поставить, сервис на 1!",NaN,NaN,−
2,"Вот так ""Мой любимый"" банк МКБ меня обманул.",NaN,NaN,−
3,Отвратительное отношение к клиентам.,Communication,Communication,−
4,"Всегда в любое время дня и ночи помогут, ответ...",Communication,Communication,+
...,...,...,...,...
19356,Никогда и ни в коем случае не открывайте счет ...,Communication,Communication,−
19357,ТИ откровенно забили на качество и развивают с...,Quality,Quality,−
19358,"Я считаю, это прорыв и лидерство финансовых ус...",NaN,NaN,+
19359,"Писал мужчина очень доходчиво, не финансовым я...",Communication,Communication,+


In [22]:
data.shape

(19361, 4)

In [23]:
data = data.dropna()

In [24]:
data.shape

(13435, 4)

In [25]:
np.array(data_to_save['sentence'])[1]

NameError: name 'data_to_save' is not defined

In [27]:
data = data.reset_index(drop=True)

In [47]:
data_to_save = {'sentence': [], 'category': []}
for i in range(data.shape[0]):
    for cat in ['1category', '2category']:
        data_to_save['category'] += [data.loc[i, cat]]
        data_to_save['sentence'] += [data.loc[i, 'sentence']]

In [48]:
data_to_save = pd.DataFrame(data_to_save)

In [49]:
for i in range(data_to_save['sentence'].shape[0]):
    if i < 26869:
        if data_to_save['sentence'][i] == data_to_save['sentence'][i+1]:
            if data_to_save['category'][i] == data_to_save['category'][i+1]:
                data_to_save['category'][i+1] = np.nan
        

In [50]:
data_ = data_to_save.dropna()

In [51]:
data_

,sentence,category
0,При этом всегда получал качественные услуги.,Communication
2,Отвратительное отношение к клиентам.,Communication
4,"Всегда в любое время дня и ночи помогут, ответ...",Communication
6,"Все время согласовывалось, всё делалось быстро.",Communication
8,Абсолютное бездействие и нежелание банка работ...,Quality
...,...,...
26860,Руководитель Ф-рин А. крайне неквалифицирован ...,Communication
26862,Никогда и ни в коем случае не открывайте счет ...,Communication
26864,ТИ откровенно забили на качество и развивают с...,Quality
26866,"Писал мужчина очень доходчиво, не финансовым я...",Communication


In [52]:
data['12category'] = data['1category'] + data['2category']

In [53]:
data['12category'].unique()

array(['CommunicationCommunication', 'QualityQuality', 'PriceQuality',
       'PricePrice', 'QualityCommunication', 'SafetySafety',
       'QualityPrice', 'PriceCommunication', 'CommunicationQuality',
       'CommunicationSafety', 'QualitySafety', 'SafetyCommunication',
       'CommunicationPrice', 'SafetyQuality', 'SafetyPrice'], dtype=object)

In [54]:
le = preprocessing.LabelEncoder()
le.fit(data_['category'].unique())

LabelEncoder()

In [55]:
target_coded = le.transform(data_['category'])

In [56]:
X_train, X_test, y_train, y_test = train_test_split(
    data_['sentence'], target_coded, test_size=0.3, random_state=42)

In [57]:
def tok(text):
    tt = TweetTokenizer()
    return tt.tokenize(text)

In [58]:
vect2 = TfidfVectorizer(ngram_range=(1, 5), tokenizer=tok)

In [59]:
bow3 = vect2.fit_transform(X_train)

D:\Python\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [60]:
bow3_test = vect2.transform(X_test)
bow3.shape

(10103, 400072)

In [61]:
scaler = MaxAbsScaler()
bow3 = scaler.fit_transform(bow3)
bow3_test = scaler.transform(bow3_test)

In [62]:
lgr = LogisticRegression(max_iter=1000, random_state=42)
lgr.fit(bow3, y_train)
pred_proba = lgr.predict_proba(bow3_test)
#score = roc_auc_score(y_test, pred_proba, multi_class='ovr')

In [63]:
score = roc_auc_score(y_test, pred_proba, multi_class='ovr')

In [64]:
score

0.7503941975661519

In [65]:
C = [10**(-n) for n in range(-3, 10)]
penalties = ['l1', 'l2']
score_0 = 0
for i in tqdm(range(len(C))):
    for penalty in penalties:
        lgr = LogisticRegression(max_iter=1000,
                                random_state=42, C=C[i],
                                penalty=penalty,
                                solver='liblinear')
        lgr.fit(bow3, y_train)
        pred_proba = lgr.predict_proba(bow3_test)
        score = roc_auc_score(y_test, pred_proba, multi_class='ovr')
        if score > score_0:
            score_0 = score
            C_ = C[i]
            p = penalty

100%|██████████████████████████████████████████████████████████████████████████████████| 13/13 [01:53<00:00,  8.70s/it]


In [66]:
C_, p

(1, 'l1')

In [67]:
lgr_best = LogisticRegression(max_iter=1000, random_state=42, C=C_, penalty=p, solver='liblinear')
lgr_best.fit(bow3, y_train)
pred_proba = lgr_best.predict_proba(bow3_test)

In [68]:
print(roc_auc_score(y_test, pred_proba, multi_class='ovr'))

0.7836880258395187


In [71]:
model = CatBoostClassifier(
                            learning_rate=0.1,
                            depth=8,
                            #l2_leaf_reg=2,
                            iterations=100,
                            random_seed=42,
                            loss_function='MultiClass',
                            verbose=True
                          )
model.fit(bow3, y_train)
pred_proba = model.predict_proba(bow3_test)
score = roc_auc_score(y_test, pred_proba, multi_class='ovr')

0:	learn: 1.3079774	total: 4.55s	remaining: 7m 30s
1:	learn: 1.2445956	total: 9.13s	remaining: 7m 27s
2:	learn: 1.1919480	total: 13.7s	remaining: 7m 23s
3:	learn: 1.1482194	total: 18.3s	remaining: 7m 18s
4:	learn: 1.1107421	total: 22.8s	remaining: 7m 13s
5:	learn: 1.0793790	total: 27.4s	remaining: 7m 8s
6:	learn: 1.0526587	total: 32s	remaining: 7m 5s
7:	learn: 1.0291216	total: 36.3s	remaining: 6m 57s
8:	learn: 1.0089143	total: 40.6s	remaining: 6m 50s
9:	learn: 0.9913909	total: 44.9s	remaining: 6m 43s
10:	learn: 0.9766834	total: 49.2s	remaining: 6m 38s
11:	learn: 0.9637453	total: 53.5s	remaining: 6m 32s
12:	learn: 0.9523552	total: 57.7s	remaining: 6m 26s
13:	learn: 0.9424254	total: 1m 2s	remaining: 6m 20s
14:	learn: 0.9338129	total: 1m 6s	remaining: 6m 15s
15:	learn: 0.9260539	total: 1m 10s	remaining: 6m 10s
16:	learn: 0.9185107	total: 1m 14s	remaining: 6m 5s
17:	learn: 0.9122695	total: 1m 19s	remaining: 6m
18:	learn: 0.9049759	total: 1m 23s	remaining: 5m 55s
19:	learn: 0.9002198	total:

In [72]:
score

0.7453910946614325